In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, scale, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from scipy.stats import pearsonr
import seaborn as sns
from pathlib import Path
import clean_functions as cf
from matplotlib.lines import Line2D
import math
import json
import geopandas as gpd
from shapely.geometry import Polygon, Point
from pyproj import CRS
import geopandas as gpd
from shapely.geometry import Polygon, Point
from shapely import wkt

current_dir = Path.cwd()
parent_dir = current_dir.parent 

In [3]:
Cal30 = pd.read_csv(f"{parent_dir}/data/calenviroscreen-3.0-results-june-2018-update.csv")
default_score = pd.read_csv(f"{parent_dir}/cleaned_data/default_score.csv")
ces_df2 = pd.read_csv(f"{parent_dir}/cleaned_data/cleaned_ces_cdc.csv")
party_data = pd.read_csv(f"{parent_dir}/data/california_state_assembly_district_parties_03_23.csv")
raw_geo_df = pd.read_csv(f'{parent_dir}/cleaned_data/geojson_data.csv')
raw_geo_df['geometry'] = raw_geo_df['geometry'].apply(wkt.loads)
geo_df = gpd.GeoDataFrame(raw_geo_df, geometry='geometry')
models = pd.read_csv(f'{parent_dir}/cleaned_data/models.csv')

In [4]:
env_exp_vars = ["Ozone","PM2.5","Diesel PM","Drinking Water","Lead","Pesticides","Tox. Release","Traffic"]
env_eff_vars = ['Cleanup Sites','Groundwater Threats','Haz. Waste','Imp. Water Bodies','Solid Waste']
ses_vars = ['Education','Linguistic Isolation','Poverty','Unemployment','Housing Burden']
#Sensitive population variables
pop_vars = ["Asthma", "Low Birth Weight", "Cardiovascular Disease"]

def calculate_dac_score(data, env_exp_vars_new=env_exp_vars, env_eff_vars_new=env_eff_vars, 
                        pop_vars_new=pop_vars, ses_vars_new=ses_vars,
                        omit_var=None, pop_weights_new=None, suffix=' Pctl',avg=None, tract=None,
                        env_eff_weight=0.5, env_exp_weight=1, ses_weight=1, pop_weight=1, adversarial=False):
    """Calculates the DAC score of the input dataframe"""
    if tract:
        #Filter by tract
        data = data.loc[data['Census Tract'] == tract]
    else:
        #Remove NA values
        data = data.dropna(subset='CES 4.0 Score')
        
    #Applying suffix
    env_exp_vars_new = [x + suffix for x in env_exp_vars_new]
    env_eff_vars_new = [x + suffix for x in env_eff_vars_new]
    ses_vars_new = [x + suffix for x in ses_vars_new]

    #Removing omitted variables
    if omit_var:
        omit_var = [x + suffix for x in omit_var]
        for var in omit_var:
            if var in env_exp_vars_new:
                env_exp_vars_new.remove(var)
            if var in env_eff_vars_new:
                env_eff_vars_new.remove(var)
            if var in ses_vars_new:
                ses_vars_new.remove(var)
            else:
                pop_vars_new.remove(var)

    #Average over components
    env_exposure = data[env_exp_vars_new].apply(np.mean, axis=1)
    env_effect = data[env_eff_vars_new].apply(np.mean, axis=1)
    ses_factors = data[ses_vars_new].apply(np.mean, axis=1)

    def weighted_mean(row, weights):
        non_na = ~np.isnan(row)
        return np.average(row[non_na], weights=weights[non_na])
        
    if pop_weights_new is None:
        if isinstance(pop_vars_new[0], list):
            pop_weights_new = [np.ones(len(group)) / len(group) for group in pop_vars_new]
        else:
            pop_weights_new = np.ones(len(pop_vars_new)) / len(pop_vars_new)
    
    if not isinstance(pop_vars_new[0], list):
        sens_pop = data[[var + suffix for var in pop_vars_new]].apply(
            lambda row: weighted_mean(row, weights=pop_weights_new), axis=1)
    #This condition likely won't happen
    else:
        sens_pop_matrix = np.zeros((len(data), len(pop_vars_new)))
        for i in range(len(pop_vars_new)):
            group_avg = data[[var + suffix for var in pop_vars_new[i]]].apply(lambda row: weighted_mean(row, weights=pop_weights_new[i]), axis=1)
            sens_pop_matrix[:, i] = group_avg
        sens_pop = np.nanmean(sens_pop_matrix, axis=1)
        
    score_df = pd.DataFrame({
        'Census Tract': data['Census Tract'],
        'County': data['County'],
        'env_exposure': env_exposure,
        'env_effect': env_effect,
        'sens_pop': sens_pop,
        'ses_factors': ses_factors,
    })

    if adversarial:
        score_df['Party'] = data['Party']
        score_df['Population Density'] = data['Population Density']
        score_df['Density Rank'] = score_df['Population Density'].rank(pct=True)*100
    
    score_df['Pollution Burden'] = score_df[['env_exposure', 'env_effect']].apply(lambda row: np.average(row, weights=[env_exp_weight, 
                                                                                                                       env_eff_weight]), axis=1)
    score_df['Pop Char'] = score_df[['sens_pop', 'ses_factors']].apply(lambda row: np.average(row, weights=[ses_weight, pop_weight]), axis=1)
    
    max_pollution_burden = max(score_df['Pollution Burden'])
    max_pop_char = max(score_df['Pop Char'])
    
    #Rescaling 
    if len(score_df.loc[score_df['Pollution Burden'] < 0].values) > 0 or len(score_df.loc[score_df['Pop Char'] < 0].values) > 0:
        scaler = MinMaxScaler(feature_range=(0,10))
        score_df['Pollution Burden MinMax'] = scaler.fit_transform(score_df['Pollution Burden'].values.reshape(-1, 1))
        score_df['Pop Char MinMax'] = scaler.fit_transform(score_df['Pop Char'].values.reshape(-1, 1))
    else:
        score_df['Pollution Burden MinMax'] = 10*score_df['Pollution Burden'] / max_pollution_burden
        score_df['Pop Char MinMax'] = 10*score_df['Pop Char'] / max_pop_char
    
    if avg == 'avg':
        score_df["Score"] = score_df[['Pollution Burden MinMax', 'Pop Char MinMax']].apply(np.mean, axis=1)
    else:
        score_df['Score'] = score_df['Pollution Burden MinMax'] * score_df['Pop Char MinMax']
        
    score_df['Percentile'] = score_df['Score'].rank(pct=True)*100
    
    def designate(x):
        if x >= 75: #Changing this to > instead of >= makes the standardized % change equal to the paper
            return 1 
        else:
            return 0
    
    for col in score_df.columns[2:]:
        score_df[col] = score_df[col].apply(lambda x: round(x, 3))

    score_df['DAC'] = score_df['Percentile'].apply(designate)


    return score_df

def get_variable_impact(df, env_eff_weight=0.5, env_exp_weight=1, ses_weight=1, pop_weight=1):
    max_env_exp = max(df['env_exposure'])
    max_env_eff = max(df['env_effect'])
    max_pop = max(df['sens_pop'])
    max_ses = max(df['ses_factors'])
    pollution_burden_weight = env_exp_weight + env_eff_weight
    population_burden_weight = ses_weight + pop_weight
    max_score = max(df['Pollution Burden'] * df['Pop Char'])
    def get_category_contributions(row):
        true_score = (row['Pollution Burden']) * row['Pop Char']
        env_eff_control = (row['env_exposure']*env_exp_weight / pollution_burden_weight) * row['Pop Char']
        env_eff_pct = round((true_score - env_eff_control) / true_score, 3) / 2
        env_exp_control = (row['env_effect']*env_eff_weight / pollution_burden_weight) * row['Pop Char']
        env_exp_pct = round((true_score - env_exp_control) / true_score, 3) / 2
        ses_control = row['Pollution Burden'] * (row['sens_pop']*pop_weight / population_burden_weight)
        ses_pct = round((true_score - ses_control) / true_score, 3) / 2
        pop_control = row['Pollution Burden'] * (row['ses_factors']*ses_weight / population_burden_weight)
        pop_pct = round((true_score - pop_control) / true_score, 3) / 2
        return [env_exp_pct, env_eff_pct, ses_pct, pop_pct]
    
    return df.apply(lambda row: get_category_contributions(row), axis=1)

def get_variable_ranks(df, tract):
    row = df.loc[df['Census Tract'] == tract]

    return [row['env_exposure'].values, row['env_effect'].values,
            row['ses_vactors'].values, row['sens_pop'].values]


def get_tract_changes(new_df, old_df=default_score):
    """Calculates number of tract changes, returning the percent of designations
    which changed between the input dataframes as well as which tracts changed"""

    old_tracts = set(old_df.loc[old_df['DAC'] == 1]['Census Tract'].values)
    new_tracts = set(new_df.loc[new_df['DAC'] == 1]['Census Tract'].values)
    removed_tracts = old_tracts.difference(new_tracts)
    added_tracts = new_tracts.difference(old_tracts)
    total_changes = removed_tracts | added_tracts
    percent_change = round(len(total_changes) / len(old_df) * 100, 2)
    changed_counties = list(set(old_df.loc[old_df['Census Tract'].isin(list(total_changes))]['County']))
    return (removed_tracts, added_tracts, total_changes, percent_change, len(total_changes), changed_counties)

def simple_dac(data,tract,  env_exp_vars_new=env_exp_vars, env_eff_vars_new=env_eff_vars, 
                        pop_vars_new=pop_vars, ses_vars_new=ses_vars,
                        omit_var=None, pop_weights_new=None, suffix=' Pctl',avg=None,
                        env_eff_weight=0.5, env_exp_weight=1, ses_weight=1, pop_weight=1):

    data = data.loc[data['Census Tract'] == tract]
        
    #Applying suffix
    env_exp_vars_new = [x + suffix for x in env_exp_vars_new]
    env_eff_vars_new = [x + suffix for x in env_eff_vars_new]
    ses_vars_new = [x + suffix for x in ses_vars_new]
    
    #Average over components
    env_exposure = data[env_exp_vars_new].apply(np.mean, axis=1)
    env_effect = data[env_eff_vars_new].apply(np.mean, axis=1)
    ses_factors = data[ses_vars_new].apply(np.mean, axis=1)

    def weighted_mean(row, weights):
        non_na = ~np.isnan(row)
        return np.average(row[non_na], weights=weights[non_na])
        
    if pop_weights_new is None:
        if isinstance(pop_vars_new[0], list):
            pop_weights_new = [np.ones(len(group)) / len(group) for group in pop_vars_new]
        else:
            pop_weights_new = np.ones(len(pop_vars_new)) / len(pop_vars_new)
    
    sens_pop = data[[var + suffix for var in pop_vars_new]].apply(
        lambda row: weighted_mean(row, weights=pop_weights_new), axis=1)
     
    score_df = pd.DataFrame({
        'Census Tract': data['Census Tract'],
        'County': data['County'],
        'env_exposure': env_exposure,
        'env_effect': env_effect,
        'sens_pop': sens_pop,
        'ses_factors': ses_factors,
    })
    
    if adversarial:
        score_df['Party'] = data['Party']
        score_df['Population Density'] = data['Population Density']
        score_df['Density Rank'] = score_df['Population Density'].rank(pct=True)*100
    
    score_df['Pollution Burden'] = score_df[['env_exposure', 'env_effect']].apply(lambda row: np.average(row, weights=[env_exp_weight, 
                                                                                                                       env_eff_weight]), axis=1)
    score_df['Pop Char'] = score_df[['sens_pop', 'ses_factors']].apply(lambda row: np.average(row, weights=[ses_weight, pop_weight]), axis=1)
    
    max_pollution_burden = max(score_df['Pollution Burden'])
    max_pop_char = max(score_df['Pop Char'])
    
    #Rescaling 
    if len(score_df.loc[score_df['Pollution Burden'] < 0].values) > 0 or len(score_df.loc[score_df['Pop Char'] < 0].values) > 0:
        scaler = MinMaxScaler(feature_range=(0,10))
        score_df['Pollution Burden MinMax'] = scaler.fit_transform(score_df['Pollution Burden'].values.reshape(-1, 1))
        score_df['Pop Char MinMax'] = scaler.fit_transform(score_df['Pop Char'].values.reshape(-1, 1))
    else:
        score_df['Pollution Burden MinMax'] = 10*score_df['Pollution Burden'] / max_pollution_burden
        score_df['Pop Char MinMax'] = 10*score_df['Pop Char'] / max_pop_char
    
    if avg == 'avg':
        score_df["Score"] = score_df[['Pollution Burden MinMax', 'Pop Char MinMax']].apply(np.mean, axis=1)
    else:
        score_df['Score'] = score_df['Pollution Burden MinMax'] * score_df['Pop Char MinMax']
        
    score_df['Percentile'] = score_df['Score'].rank(pct=True)*100
    
    def designate(x):
        if x >= 75: #Changing this to > instead of >= makes the standardized % change equal to the paper
            return 1 
        else:
            return 0
            
    score_df['DAC'] = score_df['Percentile'].apply(designate)
    return score_df



In [5]:
default_score['score_comp'] = get_variable_impact(default_score)
default_score.loc[default_score['Census Tract'] == 6019001100]['score_comp'].values[0]

[0.3455, 0.1545, 0.245, 0.255]

In [5]:
new_score_df = calculate_dac_score(ces_df2)
scaled_score_df = calculate_dac_score(ces_df2, suffix=' Scaled')
avg_score_df = calculate_dac_score(ces_df2, avg='avg')

print('The percentage of tracts that change designation from switching to z-score standardization is', 
      get_tract_changes( scaled_score_df, old_df=new_score_df)[3])
print('The percentage of tracts that change designation from switching to averaging is', 
      get_tract_changes(avg_score_df, old_df=new_score_df,)[3])

The percentage of tracts that change designation from switching to z-score standardization is 5.32
The percentage of tracts that change designation from switching to averaging is 1.41


In [6]:
#Generating models - Takes a while to run
weights_3 = [0.33, 0.5, 1, 2, 3]
weights_2 = [0.5, 1, 2]
weights_4 = np.array([0.5, .7, .9, 1, 1.2, 1.4, 1.6, 1.8, 2])
weights_eff = weights_4 / 2
models = {}
for exp in weights_4:
    for eff in weights_eff:
        for ses in weights_4:
            for pop in weights_4:
                for method in ['default', 'avg', 'Scaled']:
                    #ensuring no repeat column for default score
                    if [exp, eff, ses, pop] == [1, 0.5, 1, 1]:
                        continue
                    model_title = f"({method}) exp_w: {exp}, eff_w: {eff}, ses_w: {ses}, pop_w: {pop}"
                    if method == 'Scaled':
                        score = calculate_dac_score(ces_df2, env_exp_weight=exp, 
                                                    env_eff_weight=eff, ses_weight=ses, pop_weight=pop, suffix=f" {method}")
                        models[model_title] = score['Percentile']
                    if method == 'avg':
                        score = calculate_dac_score(ces_df2, env_exp_weight=exp, 
                                                    env_eff_weight=eff, ses_weight=ses, pop_weight=pop, avg=method)
                        models[model_title] = score['Percentile']
                    else:
                        score = calculate_dac_score(ces_df2, env_exp_weight=exp, 
                                                    env_eff_weight=eff, ses_weight=ses, pop_weight=pop)
                        models[model_title] = score['Percentile']

models_df = pd.DataFrame(models)
models_df['Default_score'] = default_score['Percentile']
models_df['Min'] = models_df.apply(lambda row: min(row), axis=1)
models_df['Max'] = models_df.apply(lambda row: max(row[:-1]), axis=1)
models_df['Median'] = models_df.apply(lambda row: np.median(row[:-2]), axis=1)
models_df['Census Tract'] = score['Census Tract']



KeyboardInterrupt: 

In [13]:
def get_messages(df, tract, method='default'):
    if method == 'default':
        cols = [x for x in df.columns[:-4] if x.split()[0] == '(default)'] + list(df.columns[-4:])
        df = df[cols]
    row = df.loc[df['Census Tract'] == tract].values[0][:-4].tolist()
    cols = df.columns[:-3]
    
    def generate_message(cols, row, func):
        extrema = func(row)
        max_ind = row.index(extrema)
        title = cols[max_ind]
        print(title)
        method = title.split()[0][1:-1]
        weights_inter = " ".join(title.split()[1:])
        print(weights_inter)
        weight_vals = weights_inter.split(',')
        exp_weight = float(weight_vals[0].split()[-1])
        eff_weight = float(weight_vals[1].split()[-1])*2
        ses_weight = float(weight_vals[2].split()[-1])
        pop_weight = float(weight_vals[3].split()[-1])
        weights = {
            'environmental exposure factors': exp_weight,
            'environmental effect factors': eff_weight,
            'socioeconomic factors': ses_weight,
            'health factors': pop_weight,
        }

        wgtfreq = {
            0.5: [],
            1: [],
            2: []
        }

        for factor, value in weights.items():
            wgtfreq[value].append(factor) 
        
        for weight, factors in wgtfreq.items():
            if weight == 1:
                regmsg = ' '
                if len(factors) == 1:
                    regmsg += f'{factors[0]}'
                elif len(factors) == 2:
                    regmsg += " as well as ".join(factors)
                elif len(factors) > 2:
                    regmsg += ", ".join(factors[:-1]) + f", and {factors[-1]}"
                else:
                    regmsg = ''
                    continue
                regmsg += ' are regularly weighted'
                print(regmsg)

            if weight == 0.5:
                downmsg = ' '
                if len(factors) == 1:
                    downmsg += f'{factors[0]}'
                elif len(factors) == 2:
                    downmsg += " as well as ".join(factors)
                elif len(factors) > 2:
                    downmsg += ", ".join(factors[:-1]) + f", and {factors[-1]}"
                else:
                    downmsg = ''
                    continue
                downmsg += ' are lowly weighted'

            if weight == 2:
                upmsg = ' '
                if len(factors) == 1:
                    upmsg = f'{factors[0]}'
                elif len(factors) == 2:
                    upmsg = " as well as ".join(factors)
                elif len(factors) > 2:
                    upmsg = ", ".join(factors[:-1]) + f", and {factors[-1]}"
                else:
                    upmsg = ''
                    continue
                upmsg += ' are highly weighted'
        if (upmsg != '' and regmsg != '' and downmsg != ''):
            finalmsg = upmsg + ',' + regmsg + ', and' + downmsg 
        elif (upmsg == '' and regmsg != ''):
            finalmsg =  regmsg + ' and' + downmsg 
        else:
            finalmsg = upmsg + ' and' + regmsg + ''+ downmsg 
        return finalmsg
    
    max_msg = basemsg = 'Your census tract scores highest when ' + generate_message(cols, row, max)
    min_msg = basemsg = 'Your census tract scores lowest when' + generate_message(cols, row, min)

    var_message = "This is due to your tract's high {} levels and {} levels, but low {} levels"
    return [max_msg, min_msg]

get_messages(models_df, 6085507302, method='None')
models_df.to_csv(f'{parent_dir}/cleaned_data/models.csv', index=False)


(avg) exp_w: 0.5, eff_w: 1, ses_w: 0.5, pop_w: 2
exp_w: 0.5, eff_w: 1, ses_w: 0.5, pop_w: 2
(default) exp_w: 1, eff_w: 0.25, ses_w: 0.5, pop_w: 0.5
exp_w: 1, eff_w: 0.25, ses_w: 0.5, pop_w: 0.5
 environmental exposure factors are regularly weighted


In [8]:
## Barchart with % disadvantaged by county data
score_df = default_score
counties = set(score_df['County'].values)
perc_disadvantaged = {}
for county in counties:
    tracts = score_df.loc[score_df['County'] == county]
    disad_prop = tracts.loc[tracts['DAC'] == 1].shape[0] / tracts.shape[0]
    perc_disadvantaged[county] = disad_prop

perc_disadvantaged


{'Mendocino': 0.0,
 'Yuba': 0.14285714285714285,
 'Orange': 0.1413793103448276,
 'San Francisco': 0.07291666666666667,
 'Alpine': 0.0,
 'Merced': 0.7551020408163265,
 'San Mateo': 0.04487179487179487,
 'Del Norte': 0.0,
 'Stanislaus': 0.43617021276595747,
 'Fresno': 0.5555555555555556,
 'Modoc': 0.0,
 'San Benito': 0.0,
 'Mariposa': 0.0,
 'Siskiyou': 0.0,
 'El Dorado': 0.0,
 'Sutter': 0.23809523809523808,
 'Los Angeles': 0.4619068350021768,
 'Tehama': 0.0,
 'Sierra': 0.0,
 'Marin': 0.0,
 'Amador': 0.0,
 'Riverside': 0.1511111111111111,
 'Tulare': 0.5974025974025974,
 'Kings': 0.56,
 'Monterey': 0.02247191011235955,
 'Santa Barbara': 0.023255813953488372,
 'Colusa': 0.0,
 'Inyo': 0.0,
 'Plumas': 0.0,
 'Nevada': 0.0,
 'Trinity': 0.0,
 'Calaveras': 0.0,
 'Tuolumne': 0.0,
 'Alameda': 0.09803921568627451,
 'San Joaquin': 0.3597122302158273,
 'San Luis Obispo': 0.0,
 'Yolo': 0.0975609756097561,
 'Glenn': 0.16666666666666666,
 'Kern': 0.4965986394557823,
 'Imperial': 0.6,
 'Contra Costa': 0.1

In [44]:
## Barchart or Piechart of racial breakdown of 
# AIAN = American Indian/Alaska Native
# NHPI = Native Hawiian/Pacific Islander
race_data = pd.read_csv(f"{parent_dir}/data/acs_race_poverty_estimates.csv")
perc_cols = [x for x in race_data.columns if x[0] == 'p' and x.find('_') == -1]
total_cols = [x for x in race_data.columns if x.find('Total') != -1]
perc_cols += ['Census Tract', 'Total']
total_cols += ['Census Tract',]
race_df = race_data[perc_cols]

new_names = {'pWhiteNH': 'White',
 'pLatine': 'Latino',
 'pBlack': 'Black',
 'pAsian': 'Asian',
 'pAIAN': 'Native American',
 'pMixed': 'Mixed',
 'pOther': 'Other',
 'pNHPI': 'Pacific Islander'}
new_names2 = {'TotalWhite': 'White',
 'TotalLatine': 'Latino',
 'TotalBlack': 'Black',
 'TotalAsian': 'Asian',
 'TotalAIAN': 'Native American',
 'TotalMixed': 'Mixed',
 'TotalOther': 'Other',
 'TotalNHPI': 'Pacific Islander'}
race_df.rename(columns=new_names, inplace=True)
for col in new_names.values():
    race_df[col] = race_df[col] * race_df['Total']
race_df = score_df.merge(race_df, how='left', on='Census Tract')
race_df.dropna(subset=['Total'], inplace=True)
yes_dac = race_df.loc[race_df['DAC'] == 1]
total_yes = np.sum(yes_dac['Total'].values)
no_dac = race_df.loc[race_df['DAC'] == 0]
total_no = np.sum(no_dac['Total'].values)
yes_dac_data = {}
no_dac_data = {}

for column in new_names2.values():
    prop_yes = sum(yes_dac[column].values) / total_yes
    prop_no = sum(no_dac[column].values) / total_no
    yes_dac_data[column] = prop_yes
    no_dac_data[column] = prop_no

yes_dac_data


C:\Users\cflor\AppData\Local\Temp\ipykernel_9488\829471658.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_df.rename(columns=new_names, inplace=True)
C:\Users\cflor\AppData\Local\Temp\ipykernel_9488\829471658.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_df[col] = race_df[col] * race_df['Total']


{'White': 0.14361040566293035,
 'Latino': 0.6631297404441105,
 'Black': 0.08406901082746061,
 'Asian': 0.08991099607644522,
 'Native American': 0.009170797089320042,
 'Mixed': 0.03508319712146944,
 'Other': 0.26649424107440534,
 'Pacific Islander': 0.0036942321838094845}

In [13]:
def get_large_changes(row):
    diff = row[-1] - row[-2]
    if diff > 50:
        return diff
    else:
        return 'No'

def get_designation_switches(row):
    if row[1] > 75 and row[-2] < 75 or row[1] < 75 and row[-1] > 75:
        return 'Yes'
    else: 
        return 'No'
        
changes = models_df.apply(get_designation_switches, axis=1)
meaningful = [x for x in changes if x != 'No']
len(meaningful)

C:\Users\cflor\AppData\Local\Temp\ipykernel_23284\611430705.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[1] > 75 and row[-2] < 75 or row[1] < 75 and row[-1] > 75:


2134

In [8]:
perc_disadvantaged = [
    { 'name': 'A', 'value': 30 },
    { 'name': 'B', 'value': 100 },
    { 'name': 'C', 'value': 50 },
    { 'name': 'D', 'value': 80 },
    { 'name': 'E', 'value': 40 }
  ]
sorted_disadvantaged = []
for county in perc_disadvantaged:
        value = county['value']
        if len(sorted_disadvantaged) == 0:
            sorted_disadvantaged.append(county)
        else:
            if value > sorted_disadvantaged[0]['value']:
                sorted_disadvantaged.insert(0, county)
                continue
            for (ind, item) in enumerate(sorted_disadvantaged):
                try:
                    if item['value'] >= value and value >= sorted_disadvantaged[ind + 1]['value']:
                        sorted_disadvantaged.insert(ind+1, county)
                        break
                except: #triggers at end of list
                    sorted_disadvantaged.insert(ind, county)

sorted_disadvantaged



[{'name': 'B', 'value': 100},
 {'name': 'D', 'value': 80},
 {'name': 'C', 'value': 50},
 {'name': 'E', 'value': 40},
 {'name': 'A', 'value': 30}]

In [266]:
#Monte carlo MC for adversarial optimization
#To increase efficacy, make more variables changeable
num_trials = 1
objective = 'density'
if objective == 'party':
    target_col = 'Party'
    target_val='Republican'
if objective == 'density':
    target_col == 'Density Rank'
    target_val = 75



num_sims = 1000

for n in range(num_trials):
    exp = np.random.uniform(1,3)
    exp = exp - 1 if exp >= 2 else exp/2
    eff = np.random.uniform(1,3)
    eff = eff - 1 if eff >= 2 else eff/2
    eff = eff/2
    ses = np.random.uniform(1,3)
    ses = ses - 1 if ses >= 2 else ses/2
    pop = np.random.uniform(1,3)
    pop = pop - 1 if pop >= 2 else pop/2
    df = calculate_dac_score(ces_df2, env_exp_weight=exp, 
                                env_eff_weight=eff, ses_weight=ses, pop_weight=pop, adversarial=True)
    if objective == 'party':
        score = df.loc[df['DAC'] == 1].loc[df[target_col] == target_val].shape[0]
    if objective == 'density':
        score = df.loc[df['DAC'] == 1].loc[df[target_col] >= target_val].shape[0]
    weights = [exp, eff, ses, df]
    max_score = score
    max_df = df
    max_weights = weights

    for n in range(num_sims):
        exp_proposal = np.random.uniform(max(1, exp-0.75),min(3, exp+0.75)) #Samples from the range +- exp, bounded by 1 and 3
        exp_proposal = exp_proposal - 1 if exp_proposal >= 2 else exp_proposal/2
        eff_proposal = np.random.uniform(max(1, eff-0.75),min(3, eff+0.75))
        eff_proposal = eff_proposal - 1 if eff_proposal >= 2 else eff_proposal/2
        eff_proposal = eff_proposal/2
        ses_proposal = np.random.uniform(max(1, ses-0.75),min(3, ses+0.75))
        ses_proposal = ses_proposal - 1 if ses_proposal >= 2 else ses_proposal/2
        pop_proposal = np.random.uniform(max(1, pop-0.75),min(3, pop+0.75))
        pop_proposal = pop_proposal - 1 if pop_proposal >= 2 else pop_proposal/2
        df_proposal = calculate_dac_score(ces_df2, env_exp_weight=exp_proposal, 
                                    env_eff_weight=eff_proposal, ses_weight=ses_proposal, pop_weight=pop_proposal,
                                    adversarial=True)
        if objective == 'party':
            score_proposal = df.loc[df['DAC'] == 1].loc[df[target_col] == target_val].shape[0]
        if objective == 'density':
            score_proposal = df.loc[df['DAC'] == 1].loc[df[target_col] >= target_val].shape[0]
        proposal_weights = [exp_proposal, eff_proposal, ses_proposal, pop_proposal]

        if score_proposal > score:
            df = df_proposal
            score = score_proposal
            weights = proposal_weights
            if score > max_score:
                max_df = df
                max_score = score
                max_weights = weights
        else:
            p_accept = score_proposal / score  * .75 #change to exponential function later
            decision = np.random.uniform(0,1)
            if decision < p_accept:
                df = df_proposal
                score = score_proposal
                weights = proposal_weights
        


In [267]:
def_df = calculate_dac_score(ces_df2)
target_col = 'Party'
target_val='Republican'
base_score = def_df.loc[def_df['DAC'] == 1].loc[def_df[target_col] == target_val].shape[0]
print(base_score, max_score)
print(max_weights)


212 222
[0.6437777413098518, 0.26923817493945273, 0.6116872406819158, 0.6910406214025838]


In [12]:
geo_df

,geometry,id,DAC,County
0,"POLYGON ((-121.87556 37.39924, -121.87274 37.3...",6085504321,No,Santa Clara
1,"POLYGON ((-121.88886 37.40758, -121.88212 37.4...",6085504410,No,Santa Clara
2,"POLYGON ((-122.00238 37.24149, -122.00593 37.2...",6085507003,Missing,Santa Clara
3,"POLYGON ((-121.98458 37.23129, -121.986 37.229...",6085507004,Missing,Santa Clara
4,"POLYGON ((-121.93167 37.29803, -121.93167 37.2...",6085502204,Missing,Santa Clara
...,...,...,...,...
9110,"POLYGON ((-117.95917 33.92458, -117.95917 33.9...",6059001303,No,Orange
9111,"POLYGON ((-117.95918 33.9282, -117.95917 33.92...",6059001304,Yes,Orange
9112,"POLYGON ((-117.95046 33.94607, -117.9505 33.93...",6059001401,Yes,Orange
9113,"POLYGON ((-122.34415 37.9674, -122.34542 37.96...",6013367200,No,Contra Costa


In [48]:
test_df = calculate_dac_score(ces_df2, suffix=' Scaled')
test_df
def categorize(col):
    if col == 1:
        return 'Yes'
    elif col == 0:
        return 'No'
    else: 
        return "Missing"
test_df2 = test_df[['Census Tract', 'DAC']].copy()
test_df2['DAC'] = test_df2['DAC'].apply(categorize)
gdf2 = geo_df.merge(test_df2, how='left', left_on='id', right_on='Census Tract'
                    ).drop(columns=['DAC_x', 'Census Tract']).rename(columns={'DAC_y': 'DAC'}
                                                                     ).fillna('Missing')
nonmiss = gdf2['DAC'].values
len([x for x in nonmiss if x != 'Missing'])
gdf2

,geometry,id,County,DAC
0,"POLYGON ((-121.87556 37.39924, -121.87274 37.3...",6085504321,Santa Clara,No
1,"POLYGON ((-121.88886 37.40758, -121.88212 37.4...",6085504410,Santa Clara,No
2,"POLYGON ((-122.00238 37.24149, -122.00593 37.2...",6085507003,Santa Clara,Missing
3,"POLYGON ((-121.98458 37.23129, -121.986 37.229...",6085507004,Santa Clara,Missing
4,"POLYGON ((-121.93167 37.29803, -121.93167 37.2...",6085502204,Santa Clara,Missing
...,...,...,...,...
9110,"POLYGON ((-117.95917 33.92458, -117.95917 33.9...",6059001303,Orange,No
9111,"POLYGON ((-117.95918 33.9282, -117.95917 33.92...",6059001304,Orange,Yes
9112,"POLYGON ((-117.95046 33.94607, -117.9505 33.93...",6059001401,Orange,No
9113,"POLYGON ((-122.34415 37.9674, -122.34542 37.96...",6013367200,Contra Costa,No


In [38]:
tdf2
len(set(tdf2['Census Tract'].values))

6776

In [35]:
tdf2 = geo_df.merge(test_df2, how='left', left_on='id', right_on='Census Tract'
                    ).fillna('Missing')
def get_mismatch(row):
    if row['DAC_y'] == 'Missing' and row['DAC_x'] != 'Missing':
        return row['id']
    else: 
        return 0

set(tdf2.apply(get_mismatch, axis=1))

{0}

In [45]:
test_df['Census Tract'].apply(str)
geo_df['id'].apply(str)
present_df = geo_df.merge(test_df2, how='inner', left_on='id', right_on='Census Tract'
                    )
missing_tracts = [x for x in test_df2['Census Tract'].values if x not in present_df['Census Tract'].values]
missing_tracts

[6019000700,
 6037543202,
 6077000100,
 6037433501,
 6037433101,
 6029002500,
 6037206032,
 6077000801,
 6029002200,
 6039000800,
 6037570301,
 6019007100,
 6037208000,
 6037532603,
 6037433504,
 6029002000,
 6037242200,
 6019000800,
 6037292000,
 6037433801,
 6037199000,
 6037433102,
 6071004900,
 6039000900,
 6037402402,
 6037206200,
 6019003400,
 6019000600,
 6071000301,
 6037433700,
 6037243000,
 6029002301,
 6037570501,
 6037572900,
 6037530601,
 6037404802,
 6037232700,
 6037502301,
 6037240900,
 6071005600,
 6071005500,
 6025012100,
 6039000502,
 6029003700,
 6107003400,
 6037572800,
 6037226700,
 6037206300,
 6037402600,
 6019001900,
 6037408501,
 6037209102,
 6037240300,
 6037600502,
 6107004101,
 6107000900,
 6037433401,
 6037104810,
 6029004800,
 6025012200,
 6037203600,
 6065042800,
 6077003802,
 6037602509,
 6037532604,
 6037199800,
 6037104320,
 6037131020,
 6037433503,
 6037536200,
 6037603001,
 6029002400,
 6037530602,
 6071008002,
 6037541400,
 6019008502,
 6037502902,

In [5]:
models

,"(default) exp_w: 0.5, eff_w: 0.25, ses_w: 0.5, pop_w: 0.5","(avg) exp_w: 0.5, eff_w: 0.25, ses_w: 0.5, pop_w: 0.5","(Scaled) exp_w: 0.5, eff_w: 0.25, ses_w: 0.5, pop_w: 0.5","(default) exp_w: 0.5, eff_w: 0.25, ses_w: 0.5, pop_w: 1","(avg) exp_w: 0.5, eff_w: 0.25, ses_w: 0.5, pop_w: 1","(Scaled) exp_w: 0.5, eff_w: 0.25, ses_w: 0.5, pop_w: 1","(default) exp_w: 0.5, eff_w: 0.25, ses_w: 0.5, pop_w: 2","(avg) exp_w: 0.5, eff_w: 0.25, ses_w: 0.5, pop_w: 2","(Scaled) exp_w: 0.5, eff_w: 0.25, ses_w: 0.5, pop_w: 2","(default) exp_w: 0.5, eff_w: 0.25, ses_w: 1, pop_w: 0.5",...,"(avg) exp_w: 2, eff_w: 1, ses_w: 2, pop_w: 1","(Scaled) exp_w: 2, eff_w: 1, ses_w: 2, pop_w: 1","(default) exp_w: 2, eff_w: 1, ses_w: 2, pop_w: 2","(avg) exp_w: 2, eff_w: 1, ses_w: 2, pop_w: 2","(Scaled) exp_w: 2, eff_w: 1, ses_w: 2, pop_w: 2",Default_score,Min,Max,Median,Census Tract
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,99.974786,100.000000,100.000000,6019001100
1,99.987393,99.987393,99.987393,99.987393,99.987393,99.987393,99.987393,99.987393,99.987393,99.987393,...,99.987393,99.987393,99.987393,99.987393,99.987393,99.987393,99.558749,100.000000,99.987393,6077000700
2,99.974786,99.974786,99.974786,99.936964,99.962179,99.936964,99.886536,99.848714,99.886536,99.974786,...,99.974786,99.974786,99.974786,99.974786,99.974786,99.974786,99.609178,99.974786,99.936964,6037204920
3,99.962179,99.962179,99.962179,99.924357,99.911750,99.924357,99.861321,99.861321,99.861321,99.962179,...,99.962179,99.962179,99.962179,99.962179,99.962179,99.962179,99.180535,99.974786,99.949571,6019000700
4,99.949571,99.949571,99.949571,99.886536,99.899143,99.886536,99.836107,99.886536,99.836107,99.936964,...,99.936964,99.936964,99.949571,99.949571,99.949571,99.949571,96.961674,99.987393,99.936964,6019000200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7927,0.063036,0.037821,0.063036,0.063036,0.025214,0.063036,0.063036,0.025214,0.063036,0.100857,...,0.037821,0.100857,0.063036,0.037821,0.063036,0.063036,0.025214,0.289965,0.063036,6081609700
7928,0.050429,0.126072,0.050429,0.050429,0.163893,0.050429,0.075643,0.201715,0.075643,0.063036,...,0.214322,0.063036,0.050429,0.126072,0.050429,0.050429,0.037821,0.794251,0.075643,6085507302
7929,0.037821,0.277358,0.037821,0.037821,0.226929,0.037821,0.037821,0.252143,0.037821,0.088250,...,0.353001,0.088250,0.037821,0.277358,0.037821,0.037821,0.025214,0.769037,0.088250,6013351200
7930,0.025214,0.012607,0.025214,0.025214,0.012607,0.025214,0.025214,0.012607,0.025214,0.025214,...,0.012607,0.025214,0.025214,0.012607,0.025214,0.025214,0.012607,0.189107,0.025214,6081609601


In [ ]:
def get_max_weights(df):
    cols = [x for x in df.columns[:-5] if x.split()[0] == '(default)'] + list(df.columns[-5:])
    df = df[cols]
    tract = int(tract)
    row = df.loc[df['Census Tract'] == tract]
    row_nominmax = row.values[0][:-5].tolist()
    cols = df.columns[:-3]
    
    def generate_message(cols, row, func):
        extrema = func(row)
        max_ind = row.index(extrema)
        title = cols[max_ind]
        method = title.split()[0][1:-1]
        weights_inter = " ".join(title.split()[1:])
        print('Weights: ', title)
        weight_vals = weights_inter.split(',')
        exp_weight = float(weight_vals[0].split()[-1])
        eff_weight = float(weight_vals[1].split()[-1])*2
        ses_weight = float(weight_vals[2].split()[-1])
        pop_weight = float(weight_vals[3].split()[-1])
        weights = {
            'environmental exposure factors': exp_weight,
            'environmental effect factors': eff_weight,
            'socioeconomic factors': ses_weight,
            'health factors': pop_weight,
        }

In [ ]:

def get_variable_impact(df, env_eff_weight=0.5, env_exp_weight=1, ses_weight=1, pop_weight=1):
    max_env_exp = max(df['env_exposure'])
    max_env_eff = max(df['env_effect'])
    max_pop = max(df['sens_pop'])
    max_ses = max(df['ses_factors'])
    pollution_burden_weight = env_exp_weight + env_eff_weight
    population_burden_weight = ses_weight + pop_weight
    max_score = max(df['Pollution Burden'] * df['Pop Char'])
    
    def get_category_contributions(row):
        true_score = (row['Pollution Burden']) * row['Pop Char']
        env_eff_control = (row['env_exposure']*env_exp_weight / pollution_burden_weight) * row['Pop Char']
        env_eff_pct = round((true_score - env_eff_control) / true_score, 3) / 2
        env_exp_control = (row['env_effect']*env_eff_weight / pollution_burden_weight) * row['Pop Char']
        env_exp_pct = round((true_score - env_exp_control) / true_score, 3) / 2
        ses_control = row['Pollution Burden'] * (row['sens_pop']*pop_weight / population_burden_weight)
        ses_pct = round((true_score - ses_control) / true_score, 3) / 2
        pop_control = row['Pollution Burden'] * (row['ses_factors']*ses_weight / population_burden_weight)
        pop_pct = round((true_score - pop_control) / true_score, 3) / 2
        return [env_exp_pct, env_eff_pct, ses_pct, pop_pct]
    
    return df.apply(lambda row: get_category_contributions(row), axis=1)

In [ ]:
tst1 = calculate_dac_score()